# Installations

In [ ]:
# Instal semua library yang diperlukan
!pip install langchain langchain-groq langchain_community langchain-core \
                     langchain-huggingface langchain_text_splitters \
                     google-search-results \
                     pandas kaggle sentence-transformers faiss-cpu ipywidgets -q

# Importing Library

In [ ]:
import os
from google.colab import userdata
import pandas as pd
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from tqdm.auto import tqdm
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from langchain_groq import ChatGroq
from langchain import hub
from langchain.agents import create_react_agent, AgentExecutor
from langchain_core.prompts import PromptTemplate


# Define LangModel & Tools



In [ ]:
# 1. Untuk Kaggle (Download Data)
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

# 2. Untuk Groq (LLM Cepat & Gratis)
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

# 3. Untuk Serper (Web Search)
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')

print("Semua API Key telah dimuat.")

# Data Sourcing & Loading

In [ ]:
DATASET_NAME = 'karkavelrajaj/amazon-sales-dataset'
ZIP_FILE = 'amazon-sales-dataset.zip'

print(f"Mengunduh dataset: {DATASET_NAME}...")

# 1. Coba unduh dataset
!kaggle datasets download -d {DATASET_NAME}

# 2. Periksa apakah file zip BERHASIL diunduh
if os.path.exists(ZIP_FILE):
    print(f"Berhasil mengunduh {ZIP_FILE}.")

    # 3. Ekstrak file
    !unzip -o -q {ZIP_FILE} # -q = quiet
    print("Dataset berhasil diekstrak (amazon.csv).")
else:
    print(f"ERROR: Gagal mengunduh {ZIP_FILE}.")
    print("Jika Anda melihat error '403 Forbidden', pastikan Anda telah 'Accept Rules' di halaman Kaggle dataset.")

# Data Preprocessing, Chunking, and Vektorization

In [ ]:
print("Memulai proses pembuatan RAG Retriever...")

try:
    # 1. Muat Data
    FILE_NAME = 'amazon.csv'
    df = pd.read_csv(FILE_NAME)

    # 2. Bersihkan & Format Data
    df_cleaned = df[['product_name', 'category', 'about_product', 'discounted_price', 'rating', 'rating_count', 'product_link']].copy()
    df_cleaned = df_cleaned.dropna(subset=['product_name', 'about_product', 'category'])
    df_cleaned.fillna('N/A', inplace=True)

    docs = []
    for _, row in df_cleaned.iterrows():
        page_content = f"Product Name: {row['product_name']}\nCategory: {row['category']}\nAbout Product: {row['about_product']}"
        metadata = {
            'product_name': str(row['product_name']),
            'discounted_price': str(row['discounted_price']),
            'rating': str(row['rating']),
            'rating_count': str(row['rating_count']),
            'product_link': str(row['product_link'])
        }
        docs.append(Document(page_content=page_content, metadata=metadata))

    print(f"Data CSV berhasil diproses menjadi {len(docs)} Dokumen.")

    # 3. Inisialisasi Embeddings (Lokal & Gratis)
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(model_name=model_name)

    # 4. Split Dokumen
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    splits = text_splitter.split_documents(docs)

    # 5. Buat Vector Store (FAISS)
    print("Memulai vektorisasi... (Ini akan memakan waktu beberapa saat)")
    vector_store = FAISS.from_documents(documents=splits, embedding=embeddings)

    # 6. Buat Retriever
    retriever = vector_store.as_retriever(search_kwargs={'k': 3}) # Ambil 3 hasil teratas

    print("\n--- Retriever RAG (Database Produk) Siap! ---")

except Exception as e:
    print(f"Terjadi error saat membuat retriever: {e}")

# Tools Setup

In [ ]:
# Tool 1: Internet Search (dari notebook Anda yang berhasil)
search_wrapper = GoogleSerperAPIWrapper()
search_tool = Tool(
    name="Internet Search",
    func=search_wrapper.run,
    description="Berguna untuk mencari informasi real-time, berita terbaru, ulasan, atau fakta umum yang tidak ada di database produk."
)

# Tool 2: Product Database (RAG kita)
# Kita bungkus 'retriever' yang sudah kita buat ke dalam format 'Tool'
product_db_tool = Tool(
    name="Product Database Search",
    func=retriever.invoke, # Agen akan memanggil retriever.invoke(query)
    description="Wajib digunakan untuk mencari informasi produk internal seperti spesifikasi teknis, deskripsi, kategori, dan harga. Gunakan ini SEBELUM mencari di internet."
)

# Gabungkan kedua tool ke dalam satu daftar
tools = [search_tool, product_db_tool]

print(f"Agent sekarang memiliki {len(tools)} tools:")
print(f"1. {tools[0].name}")
print(f"2. {tools[1].name}")

# Agent Setup

In [ ]:
# 1. Inisialisasi LLM Groq
llm = ChatGroq(
    temperature=0,
    model_name="llama-3.3-70b-versatile"
)
print("LLM Groq diinisialisasi.")

# 2. Ambil ReAct Prompt
# Ini adalah "otak" yang memberi tahu agent cara menggunakan tools
prompt = hub.pull("hwchase17/react")

# 3. Buat Agent
# Agent ini tahu cara menggunakan 'llm' dan 'tools'
agent = create_react_agent(llm, tools, prompt)
print("Agent ReAct dibuat.")

# 4. Buat Agent Executor
# Ini adalah 'wrapper' yang menjalankan agent secara berulang
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True, # 'verbose=True' akan menunjukkan "pikiran" agent
    handle_parsing_errors=True # Menangani jika LLM memberi output aneh
)
print("Agent Executor dengan Groq dan 2 tools (RAG + Web) siap.")

# Execution

In [ ]:
print("---  Memulai Chatbot E-commerce Agent ---")
print("Ketik 'q' atau 'quit' untuk keluar.")
print("-" * 50)

while True:
    # 1. Menerima input dari pengguna
    query = input("Anda: ")

    # 2. Memeriksa kondisi keluar
    if query.lower() in ['q', 'quit']:
        print("Bot: Terima kasih telah menggunakan layanan kami. Sampai jumpa!")
        break

    # 3. Menjalankan agent jika bukan perintah keluar
    print("Bot:  Berpikir...")
    try:
        # Jalankan agent executor dengan input pengguna
        result = agent_executor.invoke({"input": query})

        # 4. Cetak hasil
        print("\n" + "="*50)
        print("---  JAWABAN AKHIR AGENT ---")
        print("="*50)
        print(result['output'])
        print("="*50 + "\n") # Tambahkan spasi untuk pertanyaan berikutnya

    except Exception as e:
        print(f"\nTerjadi error saat eksekusi: {e}")
        print("Maaf, terjadi kesalahan. Silakan coba pertanyaan lain.\n")